In [3]:
import subprocess
import pip
import traceback, sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("PyQt5")
install("spacy")
install("praw")
install("googlenews")

import typing
from PyQt5 import QtGui

from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import webbrowser

import spacy
import time
import praw

from GoogleNews import GoogleNews 
import pandas as pd

class KeyWords():
    def __init__(self, post):
        super().__init__()

        nlp = spacy.load("en_core_web_sm")
        text = post

        doc = nlp(text)
        self.keywords = doc[1:len(doc) - 1]

class RedditPost():
    def __init__(self, link_url):
        super().__init__()
        CLIENT_ID = "FScNHmvZjhzvuZLg8dWw1Q"
        SECRET_KEY = "TpqOKJuzEhUExe29DAGU_378Vtp0GQ"

        reddit = praw.Reddit(client_id = CLIENT_ID,
                            client_secret = SECRET_KEY,
                            user_agent = "windows:sleuth:v1.0 (by u/Mr_Slimo)")
        
        post = reddit.submission(url = link_url)
        self.post_contents = post.selftext
        self.post_key = KeyWords(self.post_contents).keywords

# Starting screen text box (Class)
class textBox(QLineEdit):
    def __init__(self):
        super().__init__()

        # Create the text box
        self.setPlaceholderText("Enter your keywords: ")
        self.setMaxLength(200)
        self.setFixedSize(1280, 80)

        self.setStyleSheet("border-radius: 10px; padding-left: 10px")

        link_font = self.font()
        link_font.setPointSize(15)
        self.setFont(link_font)

class WarningBox(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Invalid Url")

        ok_button = QDialogButtonBox.Ok
        self.buttonBox = QDialogButtonBox(ok_button)
        self.buttonBox.accepted.connect(self.accept)

        self.layout = QVBoxLayout()
        message = QLabel("Please restart the program")
        self.layout.addWidget(message)
        self.layout.addWidget(self.buttonBox)
        self.setLayout(self.layout)

        self.setFixedSize(500, 100)

# Creates webpage drop-down menu (Class)
class dropDownMenu(QComboBox):
    def __init__(self):
        super().__init__()

        self.addItems(["Reddit", "Text Entry"])
        self.setFixedSize(200, 80)

        self.setStyleSheet("border-radius: 10px")

        website_font = self.font()
        website_font.setPointSize(15)
        self.setFont(website_font)

class ScrollLabel(QScrollArea):
    def __init__(self):
        super().__init__()

        self.setWidgetResizable(True)

        content = QWidget(self)
        self.setWidget(content)

        layout = QVBoxLayout()

        self.label = QLabel(content)

        self.label.setWordWrap(True)
        self.label.setAlignment(Qt.AlignTop)

        layout.addWidget(self.label)

    def setText(self, post):
         self.label.setText(post)


# Creates the second screen, where the articles will be displayed
class UsageScreen(QWidget):
    def __init__(self, keyword, combo_box):
        super().__init__()
        if (combo_box.currentText() == "Reddit"):
            self.reddit = RedditPost(keyword)
            self.keyword = self.reddit.post_key
            self.post = self.reddit.post_contents
        else:
            self.keyword = keyword
            self.post = keyword

        self.usage_screen_layout = QHBoxLayout()
        self.left_half = QVBoxLayout()
        self.article_list = ArticleList(self.keyword)

        self.back_button = QPushButton("Back")
        self.back_button.setFixedSize(400, 100)

        post = ScrollLabel()
        post.setText(self.post)
        
        title = QLabel("Post")
        title_font = title.font()
        title_font.setPointSize(30)
        title.setFont(title_font)

        self.left_half.addWidget(title)
        self.left_half.addWidget(post)
        self.left_half.addWidget(self.back_button)
        self.usage_screen_layout.addLayout(self.left_half)
        self.usage_screen_layout.addLayout(self.article_list)

        self.setLayout(self.usage_screen_layout)


class Article(QPushButton):
    def __init__(self, title, url, date):
        super().__init__()
        self.title = title
        self.url = url
        self.date = date

        self.setText(self.title + "\n" + self.url + "\n" + self.date)
        self.setFixedSize(1280, 80)
        self.setStyleSheet("Text-align: left")

        self.pressed.connect(self.open_web)

    def open_web(self):
        target_url = QUrl(self.url)
        QDesktopServices.openUrl(target_url)

class ArticleList(QVBoxLayout):
    def __init__(self, keyword):
        super().__init__()
        self.keyword = str(keyword)

        gNews = GoogleNews(period='365d')
        gNews.search(self.keyword)
        result = gNews.result()
        data = pd.DataFrame.from_dict(result)
        data.head(10)
        if ("img" in data):
            data = data.drop(columns=["img"])

        title = QLabel("Articles")
        title_font = title.font()
        title_font.setPointSize(30)
        title.setFont(title_font)
        title.setAlignment(Qt.AlignHCenter)
        title.setContentsMargins(0, 40, 0, 40)

        self.addWidget(title)
        for res in result:
            new_article = Article(res["title"], res["link"], res["date"])
            self.addWidget(new_article)
            
class FrontPage(QVBoxLayout):
    def __init__(self):
        super().__init__()

        user_page = QHBoxLayout()

        # Builds welcome prompt in starting screen
        self.create_welcome_prompt(self)

        # Creates the text entry box in starting screen
        self.link = textBox()
        self.website = dropDownMenu()
        self.go_button = self.goButton()

        user_page.addWidget(self.link)
        user_page.addWidget(self.website)
        user_page.addWidget(self.go_button)

        # Creates layout for starting screen
        self.addLayout(user_page)
        self.link_screen = QWidget()
        self.link_screen.setLayout(self)
        
        # Aligns user input text boxes
        user_page.setContentsMargins(50, 100, 50, 400)
        user_page.setAlignment(Qt.AlignHCenter)

    # Creates welcome prompt (function)
    def create_welcome_prompt(self, link_screen_layout):
        welcome = QLabel("Welcome to Sleuth! \n Please enter your link")
        welcome_font = welcome.font()
        welcome_font.setPointSize(50)
        welcome.setFont(welcome_font)
        
        welcome.setAlignment(Qt.AlignHCenter)
        link_screen_layout.addWidget(welcome)

        welcome.setContentsMargins(0, 180, 0, 0)
    
    # Creates go button (function)
    def goButton(self):
        go_button = QPushButton("Go!")
        go_button.setFixedSize(200, 80)

        go_button_font = go_button.font()
        go_button_font.setPointSize(15)
        go_button.setFont(go_button_font)

        go_button.setStyleSheet("border-radius: 10px; background-color: white")

        return go_button
    

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Creates window
        self.setWindowTitle("Sleuth")
        self.setFixedSize(1920, 1080)

        # Initialize main layouts
        self.layout = QStackedLayout()
        self.front_page = FrontPage()

        # Adds both tabs to the stack
        self.layout.addWidget(self.front_page.link_screen)

        # Adds stack to the application
        tabs = QWidget()
        tabs.setLayout(self.layout)
        self.setCentralWidget(tabs)

        self.run_search()
        self.run_search()
        self.run_search()
        self.run_search()

    def run_search(self):
        self.front_page.link.returnPressed.connect(lambda: self.switch_tabs(self.front_page.link, self.front_page.website))
        self.front_page.go_button.pressed.connect(lambda: self.switch_tabs(self.front_page.link, self.front_page.website))

    # Returns to previous tab
    def return_tab(self):
        self.layout.setCurrentIndex(0)
        self.layout.removeWidget(self.usage_screen)

    # Switches from starting screen to article screen
    def switch_tabs(self, link, combo_box):
        keyword = link.text()
        try:
            self.usage_screen = UsageScreen(keyword, combo_box)
            self.layout.addWidget(self.usage_screen)
            self.layout.setCurrentIndex(1)

            self.usage_screen.back_button.pressed.connect(self.return_tab)
        except:
            dialog = WarningBox()
            dialog.exec()
            self.front_page.link.disconnect()
            self.front_page.go_button.disconnect()

app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()
del app